In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import col,desc,max,row_number

In [3]:
spark = SparkSession.builder.master('local').appName('Reviewed flags of top videos').getOrCreate()

In [4]:
df_user_flags = spark.read.options(header='True', InferSchema='True').csv('user_flags.csv')
df_flag_review = spark.read.options(header='True', InferSchema='True').csv('flag_review.csv')
df_user_flags.show(10)
df_flag_review.show(10)

+--------------+-------------+-----------+-------+
|user_firstname|user_lastname|   video_id|flag_id|
+--------------+-------------+-----------+-------+
|       Richard|       Hasson|y6120QOlsfU| 0cazx3|
|          Mark|          May|Ct6BUPvE2sM| 1cn76u|
|          Gina|       Korman|dQw4w9WgXcQ| 1i43zk|
|          Mark|          May|Ct6BUPvE2sM| 1n0vef|
|          Mark|          May|jNQXAC9IVRw| 1sv6ib|
|          Gina|       Korman|dQw4w9WgXcQ| 20xekb|
|          Mark|          May|5qap5aO4i9A| 4cvwuv|
|        Daniel|         Bell|5qap5aO4i9A| 4sd6dv|
|       Richard|       Hasson|y6120QOlsfU| 6jjkvn|
|       Pauline|        Wilks|jNQXAC9IVRw| 7ks264|
+--------------+-------------+-----------+-------+
only showing top 10 rows

+-------+--------------+-------------------+----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|
+-------+--------------+-------------------+----------------+
| 0cazx3|         false|               null|            null|
| 1cn76u|   

In [144]:
##to calculate max count of flaged video based on video_id
df_flag_count = df_user_flags.groupBy('video_id').count()
df_flag_count = df_flag_count.withColumnRenamed("count", "flag_count").orderBy(f.desc("flag_count"))
df_flag_count = df_flag_count.withColumnRenamed("video_id", "video_no")
df_flag_count.createOrReplaceTempView("fb")
df_most_flagged_video = spark.sql("select video_no,flag_count,rnk from "+ " (select video_no,flag_count, row_number() OVER ( ORDER BY flag_count DESC) as rnk " +" FROM fb) tmp where rnk =1 ").selectExpr('video_no')
#df_most_flagged_video.show()

In [146]:
##filtering out the flag_id for most flagged video
df_most_flaged_video = df_user_flags.join(df_most_flagged_video, df_user_flags.video_id == df_most_flagged_video.video_no, 'inner').selectExpr('video_no','flag_id')
#df_most_flaged_video.show()

In [147]:
##joining above df with flag_review df to get the  review status as true
df_review_status = df_flag_review.join(df_most_flaged_video,df_flag_review.flag_id == df_most_flaged_video.flag_id, 'inner')
df_review_status = df_review_status.filter(df_review_status.reviewed_by_yt == 'true')
#df_review_status.show()

In [148]:
## finding the final count of reviwed_by_yt as true
df_output =  df_review_status.groupBy('video_no').count().select(col('video_no').alias('video_id'),col('count').alias('number_of_reviewd_flags'))
df_output.show()

+-----------+-----------------------+
|   video_id|number_of_reviewd_flags|
+-----------+-----------------------+
|dQw4w9WgXcQ|                      5|
+-----------+-----------------------+

